In [1]:
import pandas as pd
import plotly.express as px   # type: ignore
import psycopg2 as ps
import plotly.io as pio

# Configurar Plotly para Jupyter Notebook
pio.renderers.default = "notebook"

In [2]:
def conexion_BBDD(nombre_BBDD = "BBDD_Hoteles", usuario = "postgres", contraseña = "admin", anfitrion = "localhost", puerto = "5432"):
    conn = ps.connect(
        dbname = nombre_BBDD, # base a la que nos queremos conectar
        user = usuario,
        password = contraseña,
        host = anfitrion,
        port = puerto)
    return conn
    

# Pagina inicial

## KPIS


In [3]:
conn = conexion_BBDD("BBDD_Hoteles")

In [4]:
def grafico_hoteles(conn, tabla):
   
    cur = conn.cursor()
    # Vamos a analizar los ingresos por hotel, y el numero de reservas por hotel
    query = f""" SELECT 
	                competencia,
	                count(DISTINCT id_hotel) AS Nº_hoteles
                FROM {tabla} 
                GROUP BY competencia;
            """

    cur.execute(query)
    q = cur.fetchall()
    dataframe = pd.DataFrame(q)
    dataframe = dataframe.rename(columns = {0: "Competencia", 1: "Nº_hoteles"})
    dataframe["Competencia"] = dataframe["Competencia"].apply(lambda x: "Hoteles de la competencia" if x == True else "Hoteles del grupo")
    dataframe["Cuota de mercado"] = round(dataframe['Nº_hoteles']/29, 2)*100
    
    return dataframe

In [5]:
grafico_hoteles(conn, "hoteles")

,Competencia,Nº_hoteles,Cuota de mercado
0,Hoteles del grupo,19,66.0
1,Hoteles de la competencia,10,34.0


In [14]:
def df_clientes(conn, tabla_1, tabla_2):
   
    cur = conn.cursor()
    # Vamos a analizar los ingresos por hotel, y el numero de reservas por hotel
    query = f""" SELECT 
	                competencia,
	                count(DISTINCT r.id_cliente) AS Nº_clientes
                FROM {tabla_1} as r
	                JOIN  {tabla_2} as h ON r.id_hotel = h.id_hotel 
                GROUP BY competencia ;
            """

    cur.execute(query)
    q = cur.fetchall()
    dataframe = pd.DataFrame(q)
    dataframe = dataframe.rename(columns = {0: "Competencia", 1: "Nº_clientes"})
    dataframe["Competencia"] = dataframe["Competencia"].apply(lambda x: "Clientes de la competencia" if x == True else "Clientes del grupo")
    dataframe["Cuota clientes"] = round((dataframe['Nº_clientes']/14905)*100, 2)
    
    return dataframe

In [7]:
conn.rollback()

In [15]:
df_clientes(conn, "reservas", "hoteles")

,Competencia,Nº_clientes,Cuota clientes
0,Clientes del grupo,9785,65.65
1,Clientes de la competencia,5160,34.62


In [20]:
def recurrencia_clientes(conn, tabla, parametro = "clientes recurrentes"):
    cur = conn.cursor()
    if parametro == "clientes recurrentes":

        query = f"""
            WITH cte_recurrencia_clientes AS (
            SELECT 
                    id_cliente,
                    count(DISTINCT id_reserva) AS numero_reservas
            FROM "{tabla}"
            GROUP BY  id_cliente
                    HAVING count(DISTINCT id_reserva) > 1
            ORDER BY 2 desc
            )
            SELECT 
                    count(DISTINCT id_cliente)
            FROM cte_recurrencia_clientes;
            """
    elif parametro == "clientes no recurrentes":
            query = f"""
            WITH cte_recurrencia_clientes AS (
            SELECT 
                    id_cliente,
                    count(DISTINCT id_reserva) AS numero_reservas
            FROM "{tabla}"
            GROUP BY  id_cliente
                    HAVING count(DISTINCT id_reserva) <= 1
            ORDER BY 2 desc
            )
            SELECT 
                    count(DISTINCT id_cliente)
            FROM cte_recurrencia_clientes;
            """

    cur.execute(query)
    n_clientes = cur.fetchall()
    return n_clientes[0][0]

In [ ]:
clientes = [] 
clientes_recurrentes = recurrencia_clientes(conn, "reservas")
clientes_no_recurrentes = recurrencia_clientes(conn, "reservas", "clientes no recurrentes")

clientes.append({"Clientes recurrentes": clientes_recurrentes,
                        "Clientes no recurrentes":clientes_no_recurrentes})
df_clientes = pd.DataFrame(clientes).T.reset_index()
df_clientes = df_clientes.rename(columns = {"index": "Values",0: "Nº_clientes"})
df_clientes["Recurrencia"] = round((df_clientes['Nº_clientes']/14905)*100, 2)
df_clientes

,Values,Nº_clientes,Recurrencia
0,Clientes recurrentes,93,0.62
1,Clientes no recurrentes,14812,99.38


In [42]:
def grafico_hoteles(conn, parámetro = "mercado"):
    cur = conn.cursor()

    if parámetro == "mercado":
        query = """ SELECT 
	                    nombre_hotel,
	                    sum(precio_noche) AS "Ingresos",
	                    count(DISTINCT id_reserva) AS "Nº_reservas",
                        avg(h.valoracion) AS "Valoracion_media"
                    FROM hoteles h 
	                    JOIN reservas r ON h.id_hotel = r.id_hotel	
                    GROUP BY nombre_hotel
                    ORDER BY 2 DESC;
                    """ 
        
    elif parámetro == "grupo":
        query = """ 
                SELECT 
                    nombre_hotel,
                    sum(precio_noche),
                    count(id_reserva)
                FROM "Vista_hoteles_grupo"
                GROUP BY nombre_hotel
                ORDER BY 2 DESC; 
                """
    elif parámetro == "competencia":
        query = """ 
                SELECT 
                    nombre_hotel,
                    sum(precio_noche),
                    count(id_reserva)
                FROM "Vista_hoteles_competencia"
                GROUP BY nombre_hotel
                ORDER BY 2 DESC; 
                """
    cur.execute(query)
    q = cur.fetchall()
    dataframe = pd.DataFrame(q)
    dataframe = dataframe.rename(columns = {0: "Hotel", 1: "Ingresos", 2: "Nº reservas", 3:"Valoracion media"})
    return dataframe

In [43]:
grafico_hoteles(conn)

,Hotel,Ingresos,Nº reservas,Valoracion media
0,Hotel Monte Verde,152136.809524,550,3.10
1,Hotel Sol y Luna,149237.766793,557,3.01
2,Hotel Puerta del Cielo,147359.553728,526,3.03
3,Hotel Brisas del Mar,146970.641842,534,3.09
4,Hotel Costa Azul,146405.725926,515,3.12
5,Hotel Los Almendros,145915.665951,533,3.01
6,Hotel Mirador Real,145661.051096,533,2.98
7,Hotel Luz de Madrid,144685.485607,536,3.05
8,Palacio del Sol,144070.996450,510,2.99
9,Hotel Vista Alegre,142978.894283,511,2.94


In [44]:
def info_temporales(conn, parámetro = "mercado"):
    cur = conn.cursor()

    if parámetro == "mercado":
        query = """ 
                SELECT 
	                fecha_reserva ,
	                sum(precio_noche) AS "Ingresos_por_fecha",
	                count(DISTINCT id_reserva) AS "Nº_reservas"
                FROM reservas r 
                GROUP BY fecha_reserva ;
                """ 
        
    elif parámetro == "grupo":
        query = """ 
                SELECT 
	                fecha_reserva ,
	                sum(precio_noche) AS "Ingresos_por_fecha",
	                count(DISTINCT id_reserva) AS "Nº_reservas"
                FROM "Vista_hoteles_grupo"
                GROUP BY fecha_reserva;
                """
    elif parámetro == "competencia":
        query = """ 
                SELECT 
                    fecha_reserva ,
                    sum(precio_noche) AS "Ingresos_por_fecha",
                    count(DISTINCT id_reserva) AS "Nº_reservas"
                FROM "Vista_hoteles_competencia"
                GROUP BY fecha_reserva;
                """
    cur.execute(query)
    q = cur.fetchall()
    dataframe = pd.DataFrame(q)
    dataframe = dataframe.rename(columns = {0: "fecha_reserva", 1: "Ingresos", 2: "Nº reservas"})
    return dataframe

In [49]:
info_temporales(conn, "competencia")

,fecha_reserva,Ingresos,Nº reservas
0,2025-02-21,520101.0,5172


In [67]:
def info_clientes(conn):
    cur = conn.cursor()

    query = """ 
                SELECT 
	                concat(c.nombre, ' ', c.apellido) AS "Nombre_cliente",
	                sum(r.precio_noche) AS "Gasto_total",
	                count(DISTINCT r.id_reserva)
                FROM clientes c 
	                JOIN reservas r ON c.id_cliente = r.id_cliente 
                GROUP BY concat(c.nombre, ' ',c.apellido);
                """ 
    cur.execute(query)
    q = cur.fetchall()
    dataframe = pd.DataFrame(q)
    dataframe = dataframe.rename(columns = {0: "Cliente", 1: "Gasto", 2: "Nº reservas"})
    return dataframe   

In [68]:
conn.rollback()

In [69]:
info_clientes(conn)

,Cliente,Gasto,Nº reservas
0,Aarón Alemán,284.282963,1
1,Aarón Aramburu,279.802141,1
2,Aarón Arellano,272.402335,1
3,Aarón Azorin,269.998444,1
4,Aarón Barral,267.931359,1
...,...,...,...
14900,Zoraida Saez,277.318824,1
14901,Zoraida Salmerón,89.000000,1
14902,Zoraida Sosa,272.402335,1
14903,Zoraida Valcárcel,80.000000,1
